In [55]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
sys.path.append('../src')

from simulated_annealing import simulated_annealing
from simulated_annealing import hamiltonian_objective_function
from simulated_annealing import flip_step_function
from simulated_annealing import single_pass_optimize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Load CSV file into a NumPy array
couplings = np.genfromtxt('out\\correlation_matrix_2016_2019.graphml.csv', delimiter=',')  # Adjust parameters as needed

print(couplings.shape)

(101, 101)


In [10]:
couplings = couplings - np.identity(couplings.shape[0])

In [62]:
initial_vectors = np.random.choice([-1, 0, 1], size=(1000, 101))
valleys, costs = simulated_annealing(initial_vectors, 100, 0.99, int(1e6), 
                              lambda vecs: hamiltonian_objective_function(vecs, couplings),
                              lambda vecs: flip_step_function(vecs, num_flips=1))

Iteration 0, Current Cost: [[ 2.67596547e-01]
 [-7.91897537e-01]
 [ 6.95419893e-01]
 [-6.71362066e-01]
 [-2.32960282e+00]
 [ 9.46269442e-01]
 [ 4.11061326e-01]
 [ 6.98465779e-01]
 [-1.70885399e+00]
 [ 5.27792612e-01]
 [-1.30392845e+00]
 [-1.19864332e+00]
 [ 8.34359705e-01]
 [ 9.54316775e-01]
 [-1.83637392e+00]
 [-1.34323118e+00]
 [ 9.86745156e-01]
 [-1.15825333e+00]
 [-2.80876657e-01]
 [ 1.28503747e+00]
 [-1.09962918e+00]
 [-2.82283161e-01]
 [ 6.89434212e-01]
 [ 9.13177821e-01]
 [ 1.00089306e+00]
 [ 1.13202442e-01]
 [ 9.89397625e-01]
 [-1.05028165e+00]
 [ 7.92992693e-01]
 [-1.04389332e+00]
 [-5.22712807e-01]
 [-4.00881126e+00]
 [ 7.37403865e-02]
 [ 1.56207128e+00]
 [ 1.20769666e+00]
 [ 2.95923946e-01]
 [ 4.75660641e-01]
 [-1.36977380e+00]
 [ 1.22169082e-01]
 [ 3.20229843e-01]
 [-7.50924236e-01]
 [-2.81545991e-02]
 [ 2.43095905e+00]
 [ 1.94224160e+00]
 [ 2.96575651e-01]
 [-1.14482323e+00]
 [-6.82431635e-01]
 [ 1.04042763e+00]
 [ 1.51417499e+00]
 [ 2.24223965e-01]
 [ 5.68764789e-01]
 [ 5

c:\Users\vicvi\BeliefNetworkEvo\nbk\../src\simulated_annealing.py:65: RuntimeWarning: overflow encountered in exp
  


In [63]:
second_pass_valleys = single_pass_optimize(valleys, couplings, random_order=True)
second_pass_costs = hamiltonian_objective_function(second_pass_valleys, couplings)

In [64]:
costs

array([[-22.35984853],
       [-21.89127267],
       [-18.69392307],
       [-21.51085201],
       [-20.59121155],
       [-20.15631601],
       [-21.72209063],
       [-23.88437161],
       [-19.98842255],
       [-21.04357881],
       [-23.68438723],
       [-19.61802186],
       [-19.81286594],
       [-22.35300778],
       [-17.80793101],
       [-20.89094608],
       [-18.0569273 ],
       [-20.98233516],
       [-20.48540774],
       [-21.24502354],
       [-20.16436207],
       [-22.01997121],
       [-21.44964355],
       [-19.5420909 ],
       [-20.35332812],
       [-19.37508498],
       [-22.14436481],
       [-20.1819466 ],
       [-22.39732155],
       [-19.99894219],
       [-20.56545466],
       [-19.97016164],
       [-19.118109  ],
       [-22.92382156],
       [-19.42355055],
       [-20.71819067],
       [-20.22202391],
       [-20.48026537],
       [-20.13081512],
       [-20.61995908],
       [-21.94801027],
       [-23.42963851],
       [-21.23963294],
       [-21

In [71]:
(second_pass_costs < costs).sum()

158

In [72]:
np.unique(second_pass_valleys, axis=0).shape

(915, 101)

In [25]:
np.mean(np.sum(np.abs(valleys), axis=1)) # number of nonzero entries

98.786

In [86]:
diffs = np.sum(np.abs(second_pass_valleys[None, :, :] - second_pass_valleys[:, None, :]), axis=2) / (2 * 101)

In [87]:
diffs

array([[0.        , 0.2970297 , 0.35643564, ..., 0.6039604 , 0.63366337,
        0.18811881],
       [0.2970297 , 0.        , 0.61386139, ..., 0.7029703 , 0.81188119,
        0.30693069],
       [0.35643564, 0.61386139, 0.        , ..., 0.32673267, 0.31683168,
        0.40594059],
       ...,
       [0.6039604 , 0.7029703 , 0.32673267, ..., 0.        , 0.16831683,
        0.45544554],
       [0.63366337, 0.81188119, 0.31683168, ..., 0.16831683, 0.        ,
        0.62376238],
       [0.18811881, 0.30693069, 0.40594059, ..., 0.45544554, 0.62376238,
        0.        ]])

[0.         0.2970297  0.35643564 0.42574257 0.72277228 0.43564356
 0.67326733 0.75247525 0.38613861 0.35643564 0.73267327 0.3960396
 0.6039604  0.67326733 0.48514851 0.67326733 0.3960396  0.76237624
 0.46534653 0.44554455 0.28712871 0.36633663 0.66336634 0.43564356
 0.48514851 0.37623762 0.56435644 0.21782178 0.45544554 0.54455446
 0.45544554 0.48514851 0.63366337 0.26732673 0.35643564 0.12871287
 0.48514851 0.27722772 0.35643564 0.42574257 0.20792079 0.12871287
 0.79207921 0.51485149 0.91089109 0.88118812 0.71287129 0.54455446
 0.51485149 0.28712871 0.3960396  0.66336634 0.47524752 0.71287129
 0.79207921 0.81188119 0.25742574 0.28712871 0.45544554 0.61386139
 0.74257426 0.77227723 0.27722772 0.73267327 0.32673267 0.35643564
 0.18811881 0.03960396 0.18811881 0.55445545 0.43564356 0.72277228
 0.7029703  0.32673267 0.78217822 0.23762376 0.6039604  0.0990099
 0.69306931 0.5049505  0.66336634 0.30693069 0.58415842 0.66336634
 0.55445545 0.45544554 0.61386139 0.48514851 0.55445545 0.485148